# Create High-k Metal Gate Stack Tutorial (Simplified)

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate) using the **new simplified `create_heterostructure` function**.

This approach replaces the complex multi-step interface creation with a single function call that automatically handles strain matching and layer stacking.

We'll create a representation of the material from the [QuantumATK tutorial](https://docs.quantumatk.com/tutorials/hkmg_builder/hkmg_builder.html) without the amorphous step.

<img src="https://docs.quantumatk.com/_images/update_square_default_stack.png" alt="High-k Metal Gate Stack" width="700"/>

## Key Improvements:
- **Simplified Configuration**: Replace complex `SimpleNamespace` structures with simple lists
- **Single Function Call**: Use `create_heterostructure()` instead of manual interface creation
- **Automatic Optimization**: Built-in strain matching and supercell optimization


## 1. Configuration Parameters


In [ ]:
from types import SimpleNamespace

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    SimpleNamespace(
        # Silicon substrate
        name = "Silicon",
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=3,  # atomic layers
            # xy_supercell_matrix=[[2, 0], [0, 2]],
        ),
        termination_formula=None,  # Use default termination for substrate
    ),
    SimpleNamespace(
        # SiO2 layer
        name = "SiO2",
        slab_params=SimpleNamespace(
            miller_indices=(1, 1, 1),
            thickness=4,
        ),
        termination_formula=None,  # Use default termination for SiO2
    ),
    SimpleNamespace(
        # HfO2 layer
        name = "Hafnium.*MCL",
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=2,
        ),
        termination_formula=None,  # Use default termination for HfO2
    ),
    SimpleNamespace(
        # TiN layer
        name = "TiN",
        slab_params=SimpleNamespace(
            miller_indices=(1, 1, 1),
            thickness=5,
        ),
        termination_formula=None,  # Use default termination for TiN
    )
]

# Gaps between layers, in Angstroms
GAPS = [1.0, 1.0, 1.0]

# Vacuum thickness over the heterostructure, in Angstroms
VACUUM = 10.0

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build import MaterialWithBuildMetadata
from mat3ra.made.tools.helpers import StackComponentDict

# Get materials from Standata
crystals = [MaterialWithBuildMetadata.create(Materials.get_by_name_first_match(params.name)) for params in STRUCTURE_PARAMS]

## 2. Create the heterostructure
 ### 2.1. Create a list of dictionaries for each layer's parameters

In [ ]:
stack_component_dicts = [
    StackComponentDict(
        crystal=crystals[i],
        miller_indices=dict.slab_params.miller_indices,
        thickness=dict.slab_params.thickness,
        xy_supercell_matrix=dict.slab_params.xy_supercell_matrix if hasattr(dict.slab_params, 'xy_supercell_matrix') else None,
    ) for i, dict in enumerate(STRUCTURE_PARAMS)
]

### 2.1. Create the heterostructure
Using the `create_heterostructure` function, then flipping to match the example.

In [ ]:
from mat3ra.made.tools.operations.core.unary import rotate
from mat3ra.made.tools.helpers import create_heterostructure

heterostructure = create_heterostructure(
    stack_component_dicts= stack_component_dicts,
    gaps= GAPS,
    vacuum=VACUUM,
    use_conventional_cell=True,
    optimize_layer_supercells=True,
)

flipped_heterostructure = rotate(heterostructure, angle=180, axis=[1,0,0])

## 3. Visualize the material


In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(
    flipped_heterostructure,
    repetitions=[1, 1, 1],
    viewer="wave",
    rotation='-90x'
)


## 4. Save the material

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

flipped_heterostructure.name = "Si-SiO2-HfO2-TiN Heterostructure"
set_materials(flipped_heterostructure)
download_content_to_file(flipped_heterostructure.to_json(), "heterostructure_high_k_metal_gate.json")
